### 1 Non-Clifford + Projection onto state $\ket{1}$
- Consider circuit $U$ with one non-Clifford gate ($T$ rotation gate) and a projection onto the one state on only one qubit ($\ket{1}\bra{1}$)
- Overarching Goal: Compute largest eigenvalue of $\bra{0^m} U^{\dagger} \ket{1}\bra{1} U \ket{0^m} \implies$ finding smallest eigenvalue of $\bra{0^m} U^{\dagger} Z U \ket{0^m}$

#### Adding T-gate Conjugation Functionality
* One of the drawbacks of using stim is that it exclusively makes use of stabilizer circuits
* Therefore, need to come up with a way to incorporate the behavior of conjugating Paulis by a T-gate
* Options:
    * Define a number of shots for random cliffords. Iterating over this number of shots, randomly, instead of placing a clifford gate, simulate action of T-gate on 
    current Pauli (I believe this will be easier to implement)
        * Define number of shots to sample cliffords
        * Start with pre-determined number of $T$-gates (Start with 1 for now)
        * For one of the shots (choose random integer from 0 to num_shots - 1) apply T gate as opposed to random clifford
        * Have list of tableaus where each element of list will be a summand.
        * Apply current stabilizer tableau to each element of list
        * In the end, there should be a list of tableaus on which we can apply each to a z-gate on the first qubit (via conjugation) to get out Sum of Paulis
    * Try to download stim locally and define a T-gate so that it is symbolically represented as a sum of stabilizer tableaus.
        * It can be written in the following way:
        $$T = \bigg(\cos \bigg(\frac{\pi}{8}\bigg) - \sin \bigg(\frac{\pi}{8}\bigg)\bigg) \mathbb{I} + e^{-i\pi/4} \sqrt{2}sin \bigg(\frac{\pi}{8}\bigg) S$$

In [58]:
import stim
import numpy as np
import copy
from typing import List, Tuple

In [59]:
class PauliSummand:
    def __init__(self, phase: complex, pauli: stim.PauliString):
        self.phase = phase
        self.pauli = pauli
        self.summand = (self.phase, self.pauli)

    def get_phase(self):
        return self.phase 

    def set_phase(self, phase: complex):
        self.phase = phase

    def get_pauli(self):
        return self.pauli 

    def set_pauli(self, pauli: stim.PauliString):
        self.pauli = pauli

    def get_matrix_rep(self):
        result = np.eye(1)
        p_I = np.array([
            [1,0],
            [0,1]
        ])
        p_X = np.array([
            [0,1],
            [1,0]
        ])
        p_Y = np.array([
            [0, -1j],
            [1j, 0]
        ])
        p_Z = np.array([
            [1,0],
            [0,-1]
        ])
        pauli_dict = {'_':p_I, 'X':p_X, 'Y':p_Y, 'Z': p_Z}
        for p_op in self.pauli.__str__()[1:]:
            result = np.kron(result, pauli_dict[p_op])
        result = self.get_phase() * result
        return result
        

class PauliSum:
    def __init__(self, *argv):
        if len(argv) == 0:
            self.sum = []
        else:
            self.sum = [argv[0]]

    def __str__(self):
        str_out = ""
        for summand in self.sum:
            str_out += "[" + str(summand.get_phase()) + "" + summand.get_pauli().__str__()[1:] + "]"
            str_out += " + "
        return str_out[:len(str_out)-3]

    def apply_tableau(self, t: stim.Tableau):
        for summand in self.sum:
            new_summand = t(summand.get_pauli())
            summand.set_phase(summand.get_phase() * new_summand.sign)
            summand.set_pauli(new_summand)

    def add_to_sum(self, p):
        self.sum += p.sum
    
    def add_summand(self, s:PauliSummand):
        self.sum.append(s)

    def apply_t_gate(self, qubit_loc:int =1):
        phase_term = 1/np.sqrt(2)
        added = False
        new_sum = PauliSum()
        for summand in self.sum:
            # By default, we're applying the T-gate to the first-qubit
            if (summand.get_pauli().__str__()[qubit_loc] == '_' or summand.get_pauli().__str__()[qubit_loc] == 'Z'):
                continue 
            elif (summand.get_pauli().__str__()[qubit_loc] == 'X'):
                if (added == False):
                    new_summand = PauliSummand(phase_term * summand.get_phase(), stim.PauliString(summand.get_pauli().__str__()[0:qubit_loc] + 'Y' + summand.get_pauli().__str__()[qubit_loc + 1:]))
                    new_sum = PauliSum(new_summand)
                    summand.set_phase(summand.get_phase() * phase_term)
                    added = True
                else:
                    new_summand = PauliSummand(phase_term * summand.get_phase(), stim.PauliString(summand.get_pauli().__str__()[0:qubit_loc] + 'Y' + summand.get_pauli().__str__()[qubit_loc + 1:]))
                    #new_sum_elem = PauliSum(new_summand)
                    summand.set_phase(summand.get_phase() * phase_term)
                    new_sum.add_summand(new_summand)
            else:
                if (added == False):
                    new_summand = PauliSummand(phase_term * summand.get_phase(), stim.PauliString(summand.get_pauli().__str__()[0:qubit_loc] + 'X' + summand.get_pauli().__str__()[qubit_loc + 1:]))
                    new_sum = PauliSum(new_summand)
                    summand.set_phase(-1 * summand.get_phase() * phase_term)
                    added = True
                else:
                    new_summand = PauliSummand(phase_term * summand.get_phase(), stim.PauliString(summand.get_pauli().__str__()[0:qubit_loc] + 'X' + summand.get_pauli().__str__()[qubit_loc + 1:]))
                    #new_sum_elem = PauliSum(new_summand)
                    summand.set_phase(-1 * summand.get_phase() * phase_term)
                    new_sum.add_summand(new_summand)
        return new_sum

    def combine_terms(self):
        pass

    def compute_eig(self, num_qubits: int):
        result = np.zeros((2**num_qubits, 2**num_qubits), dtype=complex)
        for summand in self.sum:
            result += summand.get_matrix_rep()
        return np.linalg.eig(result)

    def compute_min_eig_eigvec(self, num_qubits: int):
        eigs = self.compute_eig(num_qubits)
        e_vals = eigs[0]
        e_vecs = eigs[1]
        min_pos = np.abs(e_vals).argmin()
        return (e_vals[min_pos], e_vecs[:,min_pos]) 

In [60]:
num_wit_qubits = 3 # variable 'n' in paper
# For now, what the witness qubits are initialized to be doesn't matter (I THINK)
# as we are trying to minimize Val, given some optimal input states.
wit_qubits = np.zeros(num_wit_qubits, dtype=complex)

num_anc_qubits = 0 # variable 'm' in paper
anc_qubits = np.zeros(num_anc_qubits, dtype=complex)

num_t_gates = 1 # variable 't' in paper

num_meas_qubits = 1 # variable 'k' in paper
total_qubits = num_wit_qubits + num_anc_qubits

In [44]:
def clifford_on_basis(basis_paulis: List, cliff: stim.Tableau) -> List: 
    """ 
    Apply clifford operator on all basis_paulis (Change of basis via clifford)

    Parameters:
    -----------
    basis_paulis - Current list of basis paulis for Pauli Sum
    cliff - Tableau for clifford operator that is to applied

    Returns:
    --------
    List of basis paulis under change of basis
    """
    new_basis_paulis = [cliff(p) for p in basis_paulis]
    return new_basis_paulis 

def t_gate_transform_basis(basis_paulis: List, t_gate_loc: int) -> List:
    """ 
    Transform basis so that T-gate can be appropriately applied to basis paulis.
    We want only two bases acting non-trivially at the position where the 
    T-gate is to be applied.

    Parameters:
    -----------
    basis_paulis - Current list of basis paulis for Pauli Sum 
    t_gate_loc - Qubit location where T gate is being applied

    Returns:
    --------
    Transformed basis as prescribed by Proof of Lemma 17
    """
    pauli_dict = {0: '_', 1: 'X', 2: 'Y', 3: 'Z'}
    pauli_column = [pauli_dict[p.__getitem__(t_gate_loc - 1)] for p in basis_paulis]
    print(pauli_column)
    new_basis_paulis = []
    #if (len([elem for elem in pauli_column if elem == '_']) <= 2):
    #    return basis_paulis

    for i in range(len(pauli_column)):
        x_count = 0
        x_pos = 0
        y_count = 0
        y_pos = 0
        z_count = 0
        z_pos = 0
        if (pauli_column[i] == '_'):
            new_basis_paulis.append(basis_paulis[i])
        if (pauli_column[i] == 'X'):
            if (x_count == 0):
                new_basis_paulis.append(basis_paulis[i])
                x_count += 1
                x_pos = i
            else:
                new_basis_paulis.append(basis_paulis[x_pos] * basis_paulis[i])
                x_count += 1
            
        if (pauli_column[i] == 'Y'):
            if (y_count == 0):
                new_basis_paulis.append(basis_paulis[i])
                y_count += 1
                y_pos = i
            else:
                new_basis_paulis.append(basis_paulis[y_pos] * basis_paulis[i])
                y_count += 1

        if (pauli_column[i] == 'Z'):
            if (z_count == 0):
                new_basis_paulis.append(basis_paulis[i])
                z_count += 1
                z_pos = i
            else:
                new_basis_paulis.append(basis_paulis[z_pos] * basis_paulis[i])
                z_count += 1

    if (y_count >= 1):
        new_basis_paulis[y_pos] = (new_basis_paulis[x_pos] * new_basis_paulis[z_pos]) * new_basis_paulis[y_pos]

    new_pauli_column = [pauli_dict[p.__getitem__(t_gate_loc - 1)] for p in new_basis_paulis]
    print(new_pauli_column)
    

    return new_basis_paulis

    

def t_gate_on_basis(basis_paulis: List, t_gate_loc: int, num_qubits: int) -> List: 
    """ 
    Apply T gate on all basis_paulis 

    Parameters:
    -----------
    basis_paulis - Current list of basis paulis for Pauli Sum 
    t_gate_loc - Qubit location where T gate is being applied
    num_qubits - Number of qubits

    Returns:
    --------
    List of basis paulis under application of T-gate
    """
    new_basis_paulis = t_gate_transform_basis(basis_paulis, t_gate_loc)
    print(new_basis_paulis)
    pauli_dict = {0: '_', 1: 'X', 2: 'Y', 3: 'Z'}
    pauli_column = [pauli_dict[p.__getitem__(t_gate_loc - 1)] for p in new_basis_paulis]
    for i in range(len(pauli_column)):
        if (pauli_column[i] == '_' or pauli_column[i] == 'Z'):
            continue 
        elif (pauli_column[i] == 'X'):
            new_basis = new_basis_paulis[i] * stim.PauliString((t_gate_loc - 1) * '_' + 'X' + (num_qubits - t_gate_loc) * '_')
            new_basis = new_basis * stim.PauliString((t_gate_loc - 1) * '_' + 'Y' + (num_qubits - t_gate_loc) * '_')
            new_basis_paulis.append(new_basis)
        else:
            new_basis = new_basis_paulis[i] * stim.PauliString((t_gate_loc - 1) * '_' + 'Y' + (num_qubits - t_gate_loc) * '_')
            new_basis = new_basis * stim.PauliString((t_gate_loc - 1) * '_' + 'X' + (num_qubits - t_gate_loc) * '_')
            new_basis_paulis.append(new_basis)
            new_basis_paulis[i] = new_basis_paulis[i] * -1

    return new_basis_paulis
            
           


def sim_circ(num_tot_qubits: int, circ_depth: int, num_t_gate: int) -> Tuple:
    """ 
    Calculates U^{\dag}ZU for arbitrary Clifford + T circuit with a 
    single T gate acting WLOG on the first qubit

    Parameters:
    -----------
    * num_tot_qubits - Number of total qubits in system
    * circ_depth - Number of layers of operators in our circuit
    * num_t_gate - Number of t-gates to be inserted in our circuit

    Returns:
    ---------
    pauli_sum - Instance of PauliSum class
    """

    t = stim.Tableau(num_tot_qubits)
    gates_applied = []
    basis_paulis = []
    t_gate_layer_loc = [] # List of circuit layer positions corresponding to T-gate application
    t_gate_qubit_loc = [] # Which qubit to apply the t-gate to.
    t_gate_count = 0;
    curr_layer = 0;

    pauli_z_string = "Z" + "_" * (num_tot_qubits - 1)
    z = stim.PauliString(pauli_z_string)
    basis_paulis.append(z) # First basis vector is 'Z__...'
    z_phase = 1 + 0j
    z_summand = PauliSummand(z_phase, z)
    pauli_sum = PauliSum(z_summand)

    # Create list of circuit depth positions where we would like to apply T-gate
    for _ in range(num_t_gate):
        num_to_add = np.random.randint(0, circ_depth)
        if (num_to_add in t_gate_layer_loc):
            continue
        else:
            t_gate_layer_loc.append(num_to_add)
    
    # Create list of qubit positions where we would like to apply T-gate in each layer
    for _ in range(circ_depth):
        num_to_add = np.random.randint(1, num_tot_qubits + 1)
        t_gate_qubit_loc.append(num_to_add)

    # Need to sort this array so that application of T-gates makes chronological sense with elements of 't_gate_layer_loc'
    t_gate_layer_loc = np.sort(t_gate_layer_loc) 

    for i in range(circ_depth):
        if (t_gate_count < len(t_gate_layer_loc) and i == t_gate_layer_loc[t_gate_count]):
            # Apply tableau so far on current Pauli in center
            print("T GATE " + str(t_gate_qubit_loc[curr_layer]))
            pauli_sum.apply_tableau(t)
            pauli_sum.add_to_sum(pauli_sum.apply_t_gate(t_gate_qubit_loc[curr_layer]))

            basis_paulis = t_gate_on_basis(basis_paulis, t_gate_qubit_loc[curr_layer], num_tot_qubits)

            t_gate_count += 1
            t = stim.Tableau(num_tot_qubits)
            gates_applied.append('T')
        else:
            gate_to_add = stim.Tableau.random(num_tot_qubits)
            print(gate_to_add)
            basis_paulis = clifford_on_basis(basis_paulis, gate_to_add)
            t.append(gate_to_add, list(range(num_tot_qubits)))
            gates_applied.append(gate_to_add.__str__())
            

        curr_layer += 1

    pauli_sum.apply_tableau(t)
    return (pauli_sum, basis_paulis)

In [48]:
num_layers = 20
num_t_gates = 3

circ_sum_result = sim_circ(total_qubits, num_layers, num_t_gates)
p_sum = circ_sum_result[0]
basis = circ_sum_result[1]
print(p_sum.__str__())
min_eig_eigvec = p_sum.compute_min_eig_eigvec(total_qubits)
min_eigval = min_eig_eigvec[0]
min_eigvec = min_eig_eigvec[1]
print("The minimum eigenvalue for this pauli sum is: " + str(min_eigval))
print("The associated eigenvector is: " + str(min_eigvec))
print("The basis for the above pauli summands is: " + str(basis))


+-xz-xz-
| -- -+
| ZZ YX
| ZX YY
+-xz-xz-
| +- ++
| __ YZ
| XY __
T GATE 2
['Y']
['Y']
[stim.PauliString("+YY")]
+-xz-xz-
| -+ -+
| _X ZY
| YX YY
+-xz-xz-
| ++ -+
| Z_ YZ
| ZY Y_
+-xz-xz-
| -+ ++
| YY YZ
| XZ _Y
+-xz-xz-
| -+ ++
| __ ZX
| YZ __
T GATE 2
['X', 'Y']
['X', 'Y']
[stim.PauliString("-XX"), stim.PauliString("-XY")]
+-xz-xz-
| -- --
| X_ YZ
| XY YY
+-xz-xz-
| ++ -+
| ZY _Z
| _Z ZY
+-xz-xz-
| -+ -+
| ZX YY
| YY XZ
+-xz-xz-
| -- ++
| ZY YY
| Z_ YX
+-xz-xz-
| +- -+
| _Y YZ
| YX _Y
+-xz-xz-
| +- +-
| YX _X
| X_ XY
T GATE 2
['X', 'Y', 'Y', 'X']
['X', 'Y', 'Y', 'X']
[stim.PauliString("+YX"), stim.PauliString("-_Y"), stim.PauliString("+_Y"), stim.PauliString("+YX")]
+-xz-xz-
| -- -+
| YX ZZ
| YY XZ
+-xz-xz-
| +- --
| ZX YY
| ZZ YX
+-xz-xz-
| -- ++
| YX Z_
| XX YX
+-xz-xz-
| ++ -+
| Y_ YZ
| ZY _Y
+-xz-xz-
| ++ +-
| ZX _X
| Z_ ZX
[(-0.3535533905932737+0j)_Y] + [(0.3535533905932737-0j)YZ] + [(-0.3535533905932737+0j)YZ] + [(-0.3535533905932737+0j)_Y] + [(-0.3535533905932737+0j)_Z] + [(-0